# 2025-04-14 14:00

In [1]:
import numpy as np

In [2]:
element_dtype = np.uint8
result_dtype = np.uint32

In [3]:
generator = np.random.default_rng(42)
A = generator.integers(0, 16, (500, 100), dtype=np.uint8)
B = generator.integers(0, 16, (100, 600), dtype=np.uint8)

In [4]:
def numpy_matmul(a, b):
    return np.matmul(a, b)

In [5]:
C = numpy_matmul(A, B)

In [6]:
def naive_matmul(a, b):
    n = a.shape[0]
    q = a.shape[1]
    m = b.shape[1]
    c = np.zeros((n, m), dtype=np.uint8)
    for i in range(n):
        for j in range(m):
            for k in range(q):
                c[i, j] += a[i, k] * b[k, j]
    return c

In [7]:
c = naive_matmul(A, B)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2699044813.py:9: RuntimeWarning: overflow encountered in scalar add
  c[i, j] += a[i, k] * b[k, j]


In [8]:
assert np.allclose(C, c)

# 2025-04-17 20:45

In [9]:
def byrow_matmul(a, b):
    n = a.shape[0]
    q = a.shape[1]
    m = b.shape[1]
    c = np.zeros((n, m), dtype=np.uint8)
    for i in range(n):
        for j in range(q):
            # new cell passed in: A[i, j]
            for k in range(m):
                c[i, k] += a[i, j] * b[j, k]
    return c

In [10]:
c = byrow_matmul(A, B)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/4269207039.py:10: RuntimeWarning: overflow encountered in scalar add
  c[i, k] += a[i, j] * b[j, k]


In [11]:
assert np.allclose(C, c)

In [12]:
def small_mul(a_row, b_chunk):
    # a is a row of A, cut to the portion of b
    # b is a chunk of B
    c = np.zeros(b_chunk.shape[1], dtype=np.uint8)
    for i in range(a_row.shape[0]):
        for j in range(b_chunk.shape[1]):
            c[j] += a_row[i] * b_chunk[i, j]
        # c += a_row[i] * b_chunk[i, :]
    return c

def split_matmul(a, b, x=16, y=16):
    # split matmul into small chunks of B and rows of A
    n = a.shape[0]
    q = a.shape[1]
    m = b.shape[1]
    nchunks_x = (q + x - 1) // x
    nchunks_y = (m + y - 1) // y
    c = np.zeros((n, m), dtype=np.uint8)
    for chunk_y in range(nchunks_y):
        for chunk_x in range(nchunks_x):
            # chunk of B
            b_chunk = b[
                chunk_x * x : (chunk_x + 1) * x, chunk_y * y : (chunk_y + 1) * y
            ]
            for i in range(n):
                # row of A
                a_row = a[i, chunk_x * x : (chunk_x + 1) * x]
                c_row = small_mul(a_row, b_chunk)
                for j in range(chunk_y * y, min((chunk_y + 1) * y, m)):
                    c[i, j] += c_row[j - chunk_y * y]
                # c[i, chunk_y * y : (chunk_y + 1) * y] += c_row
    return c

In [13]:
c = split_matmul(A, B)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:7: RuntimeWarning: overflow encountered in scalar add
  c[j] += a_row[i] * b_chunk[i, j]
/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:30: RuntimeWarning: overflow encountered in scalar add
  c[i, j] += c_row[j - chunk_y * y]


In [14]:
assert np.allclose(C, c)

In [15]:
c = split_matmul(A, B, x=8, y=16)
assert np.allclose(C, c)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:7: RuntimeWarning: overflow encountered in scalar add
  c[j] += a_row[i] * b_chunk[i, j]
/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:30: RuntimeWarning: overflow encountered in scalar add
  c[i, j] += c_row[j - chunk_y * y]


In [16]:
c = split_matmul(A, B, x=8, y=8)
assert np.allclose(C, c)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:7: RuntimeWarning: overflow encountered in scalar add
  c[j] += a_row[i] * b_chunk[i, j]
/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:30: RuntimeWarning: overflow encountered in scalar add
  c[i, j] += c_row[j - chunk_y * y]


In [17]:
c = split_matmul(A, B, x=32, y=32)
assert np.allclose(C, c)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:7: RuntimeWarning: overflow encountered in scalar add
  c[j] += a_row[i] * b_chunk[i, j]
/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:30: RuntimeWarning: overflow encountered in scalar add
  c[i, j] += c_row[j - chunk_y * y]


In [18]:
c = split_matmul(A, B, x=64, y=64)
assert np.allclose(C, c)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:7: RuntimeWarning: overflow encountered in scalar add
  c[j] += a_row[i] * b_chunk[i, j]
/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:30: RuntimeWarning: overflow encountered in scalar add
  c[i, j] += c_row[j - chunk_y * y]


In [19]:
c = split_matmul(A, B, x=128, y=128)
assert np.allclose(C, c)

/var/folders/f1/j2z9_swx72543ggm95tbcfzc0000gn/T/ipykernel_1390/2245387704.py:7: RuntimeWarning: overflow encountered in scalar add
  c[j] += a_row[i] * b_chunk[i, j]
